## **Train a GPT-2 model on a custom dataset**

In this notebook I am taking a pretrained GPT-2 model and fine tune it on a custom datset. I have prepared a dataset(txt file) with combining all the deep learning lab reports I have done and put it into gpt2 model for training.
The result is pretty amazing


In [1]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

TensorFlow 1.x selected.
     |████████████████████████████████| 489.6 MB 22 kB/s 
     |████████████████████████████████| 1.3 MB 34.6 MB/s 
     |████████████████████████████████| 5.8 MB 25.0 MB/s 
     |████████████████████████████████| 463 kB 45.1 MB/s 


## GPU

In [ ]:
!nvidia-smi

Tue Nov 30 15:46:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Downloading GPT-2


In [2]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 170Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 1.60Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 570Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:49, 10.0Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 281Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 2.00Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 1.94Mit/s]


## Mounting Google Drive

In [3]:
gpt2.mount_gdrive()

Mounted at /content/drive


## Uploading a Text File to be Trained to Colaboratory

In [ ]:
file_name = "DeepLearningCorpus.txt"

In [ ]:
gpt2.copy_file_from_gdrive(file_name)

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.



Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training.
*  **`run_name`**: subfolder within `checkpoint` to save the model.
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=200,
              save_every=500
              )

Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:00<00:00, 787.07it/s]

dataset has 5528 tokens
Training...


[10 | 60.83] loss=2.07 avg=2.07
[20 | 115.93] loss=0.99 avg=1.53
[30 | 170.99] loss=0.27 avg=1.11
[40 | 226.02] loss=0.08 avg=0.84
[50 | 281.01] loss=0.13 avg=0.70
[60 | 336.01] loss=0.03 avg=0.58
[70 | 391.07] loss=0.07 avg=0.51
[80 | 446.13] loss=0.04 avg=0.45
[90 | 501.20] loss=0.03 avg=0.40
[100 | 556.26] loss=0.02 avg=0.36
[110 | 611.29] loss=0.02 avg=0.33
[120 | 666.37] loss=0.02 avg=0.30
[130 | 721.43] loss=0.02 avg=0.28
[140 | 776.48] loss=0.02 avg=0.26
[150 | 831.55] loss=0.02 avg=0.24
[160 | 886.60] loss=0.02 avg=0.23
[170 | 941.64] loss=0.02 avg=0.21
[180 | 996.68] loss=0.02 avg=0.20
[190 | 1051.74] loss=0.02 avg=0.19
[200 | 1106.79] loss=0.01 avg=0.18
======== SAMPLE 1 ========
 labelled the region proposal algorithms 

Index Terms—Transfer Learning, EfficientNet, ImageNet I. INTRODUCTION 
Serious complications can occur as a result of mal- positioned lines and tubes in patients. Doctors and nurses fre- quently use checklists for placement of lifesaving equipment to ensure 

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

The checkpoint folder is copied as a `.rar` compressed file; so that I can download it and uncompress it locally.

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [6]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

In [7]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-1000


## Generate Text From The Trained Model

After trained the model or loaded a retrained model from checkpoint, we can now generate text. `generate` generates a single text from the loaded model.

In [8]:
gpt2.generate(sess, run_name='run1')

From Net Neutrality Wiki
The first article about the topic was published in 2015. This article, the third and final one of a series to understand the fundamentals of current day object detection elaborates the technical details of the Faster R-CNN detection pipeline. For a review of its predecessors, check out these summaries: Regions with CNN (R-CNN) and Fast R-CNN. 
In the R-CNN family of papers, the evolution between versions was usually in terms of computational efficiency (integrating the different training stages), reduction in test time, and improvement in performance (mAP). These net- works usually consist of — a) A region proposal algorithm to generate “bounding boxes” or locations of possible objects in the image; b) A feature generation stage to obtain features of these objects, usually using a CNN; c) A classification layer to predict which class this object belongs to; and d) A regression layer to make the coordinates of the object bounding box more precise. 
The only stan

## **Prefix**

We can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there.

We can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, we can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup.

Other optional-but-helpful parameters for `gpt2.generate`:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, we want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [9]:
gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix="Neural Network",
              nsamples=5,
              batch_size=5
              )

Neural Network (CNN) and Fast R-CNN. 
In the R-CNN family of papers, the evolution between versions was usually in terms of computational efficiency (integrating the different training stages), reduction in test time, and improvement in performance (mAP). These net- works usually consist of — a) A region proposal algorithm to generate “bounding boxes” or locations of possible objects in the image; b) A feature generation stage to obtain features of these objects, usually using a CNN; c) A classification layer to predict which class this object belongs to; and d) A regression layer to make the coordinates of the object bounding box more precise. 
The only stand-alone portion of the network left in Fast R-CNN was the region proposal algorithm. Both R-CNN and Fast R-CNN use CPU based region proposal algorithms, Eg- the Selective search algorithm which takes around 2 seconds per image and runs on CPU computation. The Faster R-CNN [3] paper fixes this by using another convolutional network 

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [10]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

## **INFERENCE**
The above examples were generated using the 124M model finetuned on a custom dataset. If the machine is capable we may try the larger models for some state-of-the-art text generation. Probably try a larger model 355M, 774M, 1558M
and finetune it for better ressults.